In [1]:
# uses conda environment gpflow6_0

# generic
import numpy as np
import pandas as pd
import xarray as xr
from itertools import product
import time

# plotting

from matplotlib import pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.lines import Line2D 
from matplotlib.patches import Circle
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

import gpflow as gpf
from gpflow.ci_utils import ci_niter, ci_range
from gpflow.utilities import print_summary

# tensorflow
import tensorflow as tf

#debug
from IPython.core.debugger import set_trace

from eu_functions import *

In [71]:
####################  Initialize parameters #######################

ice_model = "d6g_h6g_"
lith = 'l71C'
tmax = 10000
tmin = 9000
agemax = round(tmax, -3) + 100
agemin = round(tmin, -3) - 100
place = 'northsea_uk_tight'

ages = np.arange(agemin, agemax, 100)[::-1]

locs = {
        'northsea_uk': [-10, 10, 45, 59],
        'northsea_uk_tight': [-5, 10, 50, 55],
       }
extent = locs[place]

#import khan dataset
path = '../data/HOLSEA_2019_uknorthsea.csv'
df_place = import_rsls(path, tmin, tmax, extent)

# add zeros at present-day.  
nout = 50
df_place = add_presday_0s(df_place, nout)

#####################  Make xarray template  #######################

filename = '../data/xarray_template.mat'
ds_template = xarray_template(filename, ages, extent)

#####################    Load GIA datasets   #######################

ds = make_mod(ice_model, lith, ages, extent)

likelist = []
namelist = []
wrsslist = []
rmselist = []
wrmselist = []

ds_load = ds.load().chunk((-1,-1,-1)).interp(lon=ds_template.lon, lat=ds_template.lat).to_dataset()

#####################    Run Iterative GP Regression   ##################

def ds_ageselect(ds, row):
    return ds.rsl.interp(age=[row.age]).age.values[0]

for i, ds_single in ds_load.groupby('modelrun'):

    #interpolate/select priors from GIA model
    df_place['rsl_giaprior'] = df_place.apply(lambda row: ds_select(ds_single, row), axis=1)
    df_place['age_giaprior'] = df_place.apply(lambda row: ds_ageselect(ds_single, row), axis=1)
    
    #calculate residuals
    df_place['rsl_realresid'] = df_place.rsl - df_place.rsl_giaprior
    df_place['age_realresid'] = df_place.age - df_place.age_giaprior
    
    # Calculate weighted root mean squared error and weighted residual sum of squares
    df_place['wrss'] = (df_place.age_realresid/df_place.age_er)**2 + (df_place.rsl_realresid/df_place.rsl_er)**2
    
    wrss = df_place.wrss.sum()
    
    weights = df_place.rsl_er/df_place.rsl_er.sum()
    rmse = np.sqrt((df_place.rsl_realresid ** 2).sum()/len(df_place))
    wrmse = np.sqrt((df_place.rsl_realresid ** 2/weights).sum()/len(df_place))

    
    print('number of datapoints = ', df_place.shape)
    
    ##################	  RUN GP REGRESSION 	#######################
    ##################  --------------------	 ######################


    k1 = 2500
    k2 = 10000
    k3 = 100
    k4 = 6000

    iterations = 1000
    nout = 40

    ds_giapriorinterp, da_zp, ds_priorplusgpr, ds_varp, loglike, m = run_gpr(nout, iterations, ds_single, ages, k1, k2, k3, k4, df_place)
    likelist.append(loglike)
    namelist.append(ds_single.modelrun.values.tolist()[0])
    wrsslist.append(wrss)
    rmselist.append(rmse)
    wrmselist.append(wrmse)
    
df_out = pd.DataFrame({'modelrun': namelist,
             'log_marginal_likelihood': likelist,
                      'weighted residual sum of squares': wrsslist,
                      'root mean squared error': rmselist,
                      'weighted root mean squared error': wrmselist})




number of datapoints =  (47, 15)
___First optimization___
iteration 1 likelihood -74.8033
iteration 101 likelihood -30.8562
iteration 201 likelihood -20.7044
iteration 301 likelihood -19.3490
___Second optimization___
iteration 1 likelihood -76.7119
iteration 101 likelihood -42.9523
iteration 201 likelihood -27.8440
iteration 301 likelihood -19.2622
number of datapoints =  (47, 19)
___First optimization___
iteration 1 likelihood -74.1719
iteration 101 likelihood -26.6183
iteration 201 likelihood -15.7234
iteration 301 likelihood -14.0597
___Second optimization___
iteration 1 likelihood -54.8395
iteration 101 likelihood -23.1791
iteration 201 likelihood -14.5891
iteration 301 likelihood -14.0730
number of datapoints =  (47, 19)
___First optimization___
iteration 1 likelihood -74.9329
iteration 101 likelihood -31.2515
iteration 201 likelihood -21.1202
iteration 301 likelihood -19.7882
___Second optimization___
iteration 1 likelihood -74.2489


KeyboardInterrupt: 

In [72]:
df_out = pd.DataFrame({'modelrun': namelist,
             'log_marginal_likelihood': likelist,
                      'weighted residual sum of squares': wrsslist,
                      'root mean squared error': rmselist,
                      'weighted root mean squared error': wrmselist})

df_out

,modelrun,log_marginal_likelihood,weighted residual sum of squares,root mean squared error,weighted root mean squared error
0,d6g_h6g_l71C_ump2_lm10,-19.262211,35833.631215,9.314464,99.061284
1,d6g_h6g_l71C_ump2_lm15,-14.072953,45456.355995,9.854897,110.469308


In [4]:

# path_gen = f'output/{place}_{ice_model}{ages[0]}_{ages[-1]}'
# da_zp.to_netcdf(path_gen + '_dazp')
# ds_giapriorinterp.to_netcdf(path_gen + '_giaprior')
# ds_priorplusgpr.to_netcdf(path_gen + '_posterior')
# ds_varp.to_netcdf(path_gen + '_gpvariance')

#store log likelihood in dataframe
df_out = pd.DataFrame({'modelrun': ds_single.modelrun.values.tolist(),
                 'log_marginal_likelihood': likelist})

#     writepath = f'output/{path_gen}_loglikelihood'
#     df_out.to_csv(writepath, index=False)

            

___First optimization___
iteration 1 likelihood -73.7814
iteration 101 likelihood -24.1490
iteration 201 likelihood -14.7302
iteration 301 likelihood -13.5295
___Second optimization___
iteration 1 likelihood -72.7327
iteration 101 likelihood -49.4884
iteration 201 likelihood -38.6474
iteration 301 likelihood -33.1363
iteration 401 likelihood -31.9117
iteration 501 likelihood -27.4243
iteration 601 likelihood -26.8132


name,class,transform,prior,trainable,shape,dtype,value
GPR_new.kernel.kernels[0].kernels[0].variance,Parameter,Sigmoid,,True,(),float64,1.1228295889719095
GPR_new.kernel.kernels[0].kernels[0].lengthscales,Parameter,Sigmoid,,True,(),float64,418.4078309782992
GPR_new.kernel.kernels[0].kernels[1].variance,Parameter,Sigmoid,,True,(),float64,0.5922018035247756
GPR_new.kernel.kernels[0].kernels[1].lengthscales,Parameter,Sigmoid,,True,(),float64,96490.97342368455
GPR_new.kernel.kernels[1].kernels[0].variance,Parameter,Sigmoid,,True,(),float64,0.11441389564284263
GPR_new.kernel.kernels[1].kernels[0].lengthscales,Parameter,Sigmoid,,True,(),float64,403.6711519834319
GPR_new.kernel.kernels[1].kernels[1].variance,Parameter,Sigmoid,,True,(),float64,0.07101108354035589
GPR_new.kernel.kernels[1].kernels[1].lengthscales,Parameter,Sigmoid,,True,(),float64,9212.52494980958
GPR_new.kernel.kernels[2].variance,Parameter,Sigmoid,,True,(),float64,0.01000112557452111
GPR_new.likelihood.variance,Parameter,Softplus + Shift,,True,"(47,)",float64,"[3.13821096e-01, 3.39499351e-04, 1.72677730e-04..."


PermissionError: [Errno 13] Permission denied: b'/Users/rogercreel/ws/EU_LIG/rcc_notebooks/output/northsea_uk_tight_d6g_h6g_10000_8900_dazp'

In [73]:
ice_model

'd6g_h6g_'

In [5]:
df_out = pd.DataFrame({'modelrun': ds_single.modelrun.values.tolist(),
                 'log_marginal_likelihood': likelist})
df_out

,modelrun,log_marginal_likelihood
0,d6g_h6g_l71C_ump4_lm50,-26.813227


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 12), subplot_kw=dict(projection=ccrs.LambertConformal(central_longitude=0)))

ax.coastlines(resolution='50m')
# ax.set_extent([-10, 10, 45, 60])
ax.set_extent([-5, 10, 50, 55])

scat = ax.scatter(df_place.lon, df_place.lat, s=80, c = df_place.age, 
                  cmap='viridis', edgecolor='k', linewidths=1, 
                  transform=ccrs.PlateCarree(), zorder=5)

cbaxes = inset_axes(ax, width="40%", height="3%", loc=4, borderpad=6,) 
cbar = fig.colorbar(scat, ax=ax, cax=cbaxes, shrink=.5,
                    label='Age (years)', 
                   orientation='horizontal')

ax.set_title(f'{len(df_place)} RSL data, {place} {tmax}-{tmin} yrs');

# fig.savefig('/Users/rogercreel/Desktop/dataplot_fennoscandia')